In [1]:
# MAIN
# packages urllib, beatiful soup, html5, pandas
#
# from webtableparserpreviousversion import WebTableParser
from webtableparser import WebTableParser
#

In [2]:
# Sites 1 e 2 for testing (remove comments)
'''site1 = WebTableParser()
site1.create('https://calculador.com.br/tabela/indice/IGP-M',
             'table table-bordered table-striped table-hover table-fixed mb-0')
table1 = site1.capture()
df1 = site1.parse(table1)
print(df1)'''

"site1 = WebTableParser()\nsite1.create('https://calculador.com.br/tabela/indice/IGP-M',\n             'table table-bordered table-striped table-hover table-fixed mb-0')\ntable1 = site1.capture()\ndf1 = site1.parse(table1)\nprint(df1)"

In [3]:
# Site by webtableparserpreviousversion module
'''site2 = WebTableParser('https://pt.wikipedia.org/wiki/Lista_de_capitais_do_Brasil_por_%C3%A1rea',
                       'wikitable sortable')
table2 = site2.capture()
df2 = site2.parse(table2)
print(df2)'''

"site2 = WebTableParser('https://pt.wikipedia.org/wiki/Lista_de_capitais_do_Brasil_por_%C3%A1rea',\n                       'wikitable sortable')\ntable2 = site2.capture()\ndf2 = site2.parse(table2)\nprint(df2)"

In [4]:
# site capture and parsing
site = WebTableParser()
site.create('https://www.fundsexplorer.com.br/ranking',
            'table table-hover')
table = site.capture()
df = site.parse(table)
print(df)
#
# Now you have to process the dataframe, "clean and organize" it!

    CÃ³digodo fundo                 Setor PreÃ§o Atual Liquidez DiÃ¡ria  \
0            FIVN11             Shoppings      R$ 4,00           1570.0   
1            BZLI11  TÃ­tulos e Val. Mob.     R$ 14,90           6123.0   
2            XTED11    Lajes Corporativas      R$ 7,32           3950.0   
3            ALMI11    Lajes Corporativas  R$ 1.224,67            160.0   
4            DOMC11    Lajes Corporativas    R$ 439,00             42.0   
..              ...                   ...          ...              ...   
186          XPIN11                Outros    R$ 115,87          13341.0   
187          XPML11             Shoppings    R$ 111,30          54889.0   
188          ALZR11            LogÃ­stica    R$ 132,98          14439.0   
189          FCFL11                Outros    R$ 118,00            520.0   
190          PVBI11    Lajes Corporativas     R$ 99,25          20921.0   

    Dividendo DividendYield DY (3M)Acumulado DY (6M)Acumulado  \
0     R$ 0,00         0,00%       